_unchecked_

# CNTK 302 部分: pretrained 超分辨率模型的评价

_unchecked_

<p><strong>贡献的</strong><a href="https://www.linkedin.com/in/borna-vukorepa-32a35283/">博尔纳 Vukorepa</a>2017年10月30日</p>
<h2>介绍</h2>
<p><p>CNTK 302 教程的目的是首先让用户熟悉图像的超分辨率问题, 并展示如何使用 CNTK 的工具来训练和评估执行图像超分辨率的模型。这本笔记本是一个指导, 以试验预先训练的超分辨率 CNTK 模型。关于如何准备数据和培训模型的教程载于笔记本 CNTK 302B。建议用户在继续之前完成教程 CNTK 206。对卷积神经网络的一些熟悉也被建议。</p>
<p>假设您获得了低分辨率 (例如, 200 x 200 像素) 的图像, 并且希望使其变大 (例如, 400 x 400 像素)。如果正确完成, 添加更多像素可以向图像添加更多细节 (通常是高频内容)。使用一些明显的方法, 如次插值, 细节将丢失导致模糊不清图像。</p>
<p>此问题通常称为<b>单个图像超分辨率 (SISR)</b>问题, 并且存在许多解决它的方法。到目前为止, 已显示出最佳效果的方法包括深学习和卷积神经网络。下面是一些讨论<b>SISR</b>的文章的链接。其中提到的一些方法将用于本研究。</p></p>
<p><a href="https://arxiv.org/pdf/1608.00367.pdf">加速超分辨率卷积神经网络</a></p>
<p><a href="http://cvlab.cse.msu.edu/pdfs/Tai_Yang_Liu_CVPR2017.pdf">基于深度递归残差网络的图像超分辨率</a></p>
<p><a href="https://arxiv.org/pdf/1511.04587.pdf">用非常深的卷积网络精确的图像超分辨率</a></p>
<p><a href="https://arxiv.org/pdf/1609.04802.pdf">基于生成对抗网络的照片逼真单图像超分辨率</a></p>
<p><a href="https://arxiv.org/pdf/1606.01299.pdf">RAISR: 快速准确的图像超分辨率</a></p>
<p>由于最近的深入学习结果 (如甘斯), 我们的目标是探索图像超分辨率的空间, 同时查看<b>gan 和非 gan</b>方法。所有的工作都是为推广系数2。对于其他因素, 数据集的准备和方法是完全 analoguous 的。</p>

_unchecked_

<h2>动机</h2>
<p><p>由于超分辨率模型可以应用于大量的问题, 因此对<b>SISR</b>的研究对社区有很大的用处。</p>
<p>例如, 超分辨率模型可以应用于医学等领域, 在那里他们可以帮助医生阅读 x 光扫描和类似的。同样, 他们可能会对警方从摄像头拍摄的照片中辨认罪犯很有用。然而, 很重要的一点是, 注意到每个潜在的应用程序都需要专门为该问题设计的特定培训集。在这里, 我们将不关注任何特定的问题<b>SISR</b> , 因此我们将不需要这些特殊的数据集。</p></p>

_unchecked_

<h2>评估和过滤器</h2>
<p><p>我们训练了四超分辨率模型: <b>VDSR</b>、 <b>DRNN</b>、 <b>SRResNet</b>、 <b>SRGAN</b>。我们将简单地描述他们的工作原理。在 CNTK 302B 中, 我们将更详细地描述它们, 以及他们的体系结构和培训程序。</p>
<ul>
<li><b>VDSR</b>模型以模糊的 64 x 64 模糊图像补丁作为输入, 并输出模糊修补程序和原始清晰版本的预测差异。在将该差异添加到模糊的补丁之后, 我们得到了我们的预测, 这也是 64 x 64。这个想法是在运行模型之前用次插值对目标图像进行高级化, 因此在我们通过补丁评估补丁之后, 我们将得到一个更清晰的提升图像版本。相关的纸张可以<a href="https://arxiv.org/pdf/1511.04587.pdf">在此处</a>找到。</li>
<li><b>DRNN</b>模型的工作方式与<b>VDSR</b>相同, 但它具有不同的体系结构。相关的纸张可以<a href="http://cvlab.cse.msu.edu/pdfs/Tai_Yang_Liu_CVPR2017.pdf">在此处</a>找到。</li>
<li><b>SRResNet</b>和<b>SRGAN</b>采取 112 x 112 原始修补程序, 并将其高级到模型中的预测 224 x 224 版本 (没有涉及推广的预置)。两者都有相同的体系结构, 但损失函数是不同的, 我们将在302B 教程中看到。可以在<a href="https://arxiv.org/pdf/1609.04802.pdf">此处</a>找到描述这两种方法的文件。</li>
</ul>
<p>通常, <b>VDSR</b>和<b>DRNN</b>倾向于给我们提供更清晰的图像, 但它们有时会在生成的图像中引入次要的工件, 使它们看起来不那么令人信服。另一方面, <b>SRResNet</b>和<b>SRGAN</b>倾向于给我们更少的清晰度, 但也没有工件。</p>
<p>这给我们的想法, 以某种方式结合这些模型, 希望我们将能够得到更好的结果, 而不是仅仅应用我们的四基本模型之一。请注意, <b>VDSR</b>和<b>DRNN</b>不会更改图像尺寸, 因为所有推广都是在预处理中完成的。这就是为什么我们可以用它们来进一步清理那些作为四基本模型结果的图像。</p>
<p>我们将调用这样的模型组合<b>筛选器</b>。<b>filter</b>由两个模型组成, 一个接一个地应用。推广将使用第一个模型完成, 因此第二个模式可以是<b>VDSR</b>或<b>DRNN</b>。</p>
<p>例如, 一个筛选器可能由<b>SRGAN</b>和<b>VDSR</b>组成。首先, 我们将<b>SRGAN</b>应用于起始图像, 然后对结果进行<b>VDSR</b> , 但不进行次插值预处理, 因为<b>SRGAN</b>已经提升该图像。由于<b>VDSR</b>被训练为清除模糊的图像, 因此我们可以期待更多的改进, 因为<b>SRGAN</b>在推广比次插值时明显更好。</p>
<p>同样, 我们可以将<b>VDSR</b>与自身合并。首先, 我们将使用次插值对起始图像进行高级处理, 然后在<b>VDSR</b> (第二次没有预推广) 的行中将其清除两次。该滤波器具有很高的清晰度, 但有较高的工件出现的可能性, 可以降低图像质量。</p>
<p>我们将需要启用对任意大小图像的评估。请记住, 我们的模型期望输入具有固定的尺寸。这是很容易修复的, 通过评估图像补丁补丁。由于边界像素有时会被精确地预测, 因此我们会略微重叠这些补丁, 并且不会将接近边界的像素带入该帐户。这将确保每个像素被预测为某个补丁的非边界像素, 除了那些真正位于图像边界上的。我们必须小心, 并记住, 一些模型只是学习残差图像。我们也不能忘记, 以255的规模回结果。</p>
<p>我们从一些基本的评估配置开始。</p></p>

In [1]:
import cntk as C
from PIL import Image
import os
import numpy as np
import urllib
from scipy.misc import imsave

try:
    from urllib.request import urlretrieve, urlopen
except ImportError: 
    from urllib import urlretrieve, urlopen

try:
    C.device.try_set_default_device(C.device.gpu(0))
except:
    print("GPU unavailable. Using CPU instead.")

In [2]:
# Determine the data path for testing
# Check for an environment variable defined in CNTK's test infrastructure
envvar = 'CNTK_EXTERNAL_TESTDATA_SOURCE_DIRECTORY'
def is_test(): return envvar in os.environ

if is_test():
    test_data_path_base = os.path.join(os.environ[envvar], "Tutorials", "data")
    test_data_dir = os.path.join(test_data_path_base, "BerkeleySegmentationDataset")
    test_data_dir = os.path.normpath(test_data_dir)
    

#prefer our default path for the data
data_dir = os.path.join("data", "BerkeleySegmentationDataset")

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
#folder with images to be evaluated
example_folder = os.path.join(data_dir, "example_images")
if not os.path.exists(example_folder):
    os.makedirs(example_folder)

#folders with resulting images
results_folder = os.path.join(data_dir, "example_results")
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

#names of used models
model_names = ["VDSR", "DRNN", "SRResNet", "SRGAN"]

#output dimensions of models respectively (assumed that output is a square)
output_dims = [64, 64, 224, 224]

_unchecked_

上面的评估算法在函数 `evaluate` 中实现。有关每个步骤的详细信息, 请参见代码注释。

In [3]:
#filename - relative path of image being processed
#model - the model for super-resolution
#outfile - relative path of the image which will be saved

#output_dims - dimensions of current model output image
#            - it is assumed that model output image is a square

#pre_upscale - if True, image will be upscaled by a specified factor with bicubic interpolation at the start
#            - the resulting image then replaces the original one in the next operations
#            - if False, that step is skipped
#            - this should be set on True for models which are clearing up the image and don't make upscaling by themselves

#clear_up - if True, the forwarded image will be cleared up by the model and not upscaled
#         - this is important to know because step variables are different then (see code)
#         - notice that we exit the function if pre_upscale is True and clear_up false because if image was pre-upscaled,
#           it should be cleared up afterwards

#residual_model - is the model learning residual image only (the difference between blurry and original patch)?
#               - if true, residual is added to the low resolutin image to produce the result
#               - otherwise, we only need to scale back the result (see code below)
def evaluate(filename, model, outfile, output_dims, pre_upscale = False, clear_up = False, residual_model = False):
    img = Image.open(filename)
    
    #upscaling coefficient
    coef = 2
    
    #at each step, we will evaluate subpatch (x : x + range_x, y : y + range_y) of original image
    #patch by patch, we will resolve the whole image
    range_x = output_dims // coef
    range_y = output_dims // coef
    
    #how many bounding pixels from resulting patch should be excluded?
    #this is important because boundaries tend to be predicted less accurately
    offset = output_dims // 10
    
    #after we evaluate a subpatch, how much we move down/right to get the next one
    #we subtract offset to cover those pixels which were boundary in the previous subpatch
    step_x = range_x - offset
    step_y = range_y - offset
    
    #situation which should not occur, if we need preprocess, we will need to clear up the result
    if((pre_upscale) and (not clear_up)):
        print("Pre-magnified image is not being cleared up.")
        return
    
    #pre-magnify picture if needed
    if(pre_upscale):
        img = img.resize((coef * img.width, coef * img.height), Image.BICUBIC)
    
    #if the current image is being cleared up with no further uspcaling,
    #set coef to 1 and other parameters accordingly
    if(clear_up):
        result = np.zeros((img.height, img.width, 3))
        range_x = output_dims
        range_y = output_dims
        step_x = range_x - 2 * offset
        step_y = range_y - 2 * offset
        coef = 1
    #otherwise, set result to be coef (2 by default) times larger than image
    else:
        result = np.zeros((coef * img.height, coef * img.width, 3))
    
    rect = np.array(img, dtype = np.float32)
    
    #if the image is too small for some models to work on it, pad it with zeros
    if(rect.shape[0] < range_y):
        pad = np.zeros((range_y - rect.shape[0], rect.shape[1], rect.shape[2]))
        rect = np.concatenate((rect, pad), axis = 0).astype(dtype = np.float32)
        
    if(rect.shape[1] < range_x):
        pad = np.zeros((rect.shape[0], range_x - rect.shape[1], rect.shape[2]))
        rect = np.concatenate((rect, pad), axis = 1).astype(dtype = np.float32)
    
    x = 0
    y = 0
    
    #take subpatch by subpatch and resolve them to get the final image result
    while(y < img.width):
        x = 0
        while(x < img.height):
            rgb_patch = rect[x : x + range_x, y : y + range_y]
            rgb_patch = rgb_patch[..., [2, 1, 0]]
            rgb_patch = np.ascontiguousarray(np.rollaxis(rgb_patch, 2))
            pred = np.squeeze(model.eval({model.arguments[0] : [rgb_patch]}))
            
            img1 = np.ascontiguousarray(rgb_patch.transpose(2, 1, 0))
            img2 = np.ascontiguousarray(pred.transpose(2, 1, 0))
            
            #if model predicts residual image,
            #scale back the prediction and add to starting patch
            #otherwise just scale back
            if(residual_model):
                img2 = 255.0 * img2 + img1
            else:
                img2 = pred.transpose(2, 1, 0)
                img2 = img2 * 255.0
                
            # make sure img2 is C Contiguous as we just transposed it
            img2 = np.ascontiguousarray(img2)
            #make sure no pixels are outside [0, 255] interval
            for _ in range(2):
                img2 = C.relu(img2).eval()
                img2 = np.ones(img2.shape) * 255.0 - img2
            
            rgb = img2[..., ::-1]
            patch = rgb.transpose(1, 0, 2)
            
            #fill in the pixels in the middle of the subpatch
            #don't fill those within offset range to the boundary
            for h in range(coef * x + offset, coef * x + output_dims - offset):
                for w in range(coef * y + offset, coef * y + output_dims - offset):
                    for col in range(0, 3):
                        result[h][w][col] = patch[h - coef * x][w - coef * y][col]
            
            #pad top
            if(x == 0):
                for h in range(offset):
                    for w in range(coef * y, coef * y + output_dims):
                        for col in range(0, 3):
                            result[h][w][col] = patch[h][w - coef * y][col]
            
            #pad left
            if(y == 0):
                for h in range(coef * x, coef * x + output_dims):
                    for w in range(offset):
                        for col in range(0, 3):
                            result[h][w][col] = patch[h - coef * x][w][col]
                  
            #pad bottom
            if(x == img.height - range_x):
                for h in range(coef * img.height - offset, coef * img.height):
                    for w in range(coef * y, coef * y + output_dims):
                        for col in range(0, 3):
                            result[h][w][col] = patch[h - coef * x][w - coef * y][col]
            
            #pad right                
            if(y == img.width - range_y):
                for h in range(coef * x, coef * x + output_dims):
                    for w in range(coef * img.width - offset, coef * img.width):
                        for col in range(0, 3):
                            result[h][w][col] = patch[h - coef * x][w - coef * y][col]
            
            #reached bottom of image
            if(x == img.height - range_x):
                break
            #next step by x, we must not go out of bounds
            x = min(x + step_x, img.height - range_x)
        
        #reached right edge of image
        if(y == img.width - range_x):
            break
        #next step by y, we must not go out of bounds
        y = min(y + step_y, img.width - range_x)
        
    result = np.ascontiguousarray(result)
    
    #save result
    imsave(outfile, result.astype(np.uint8))

_unchecked_

现在我们加载我们的 pretrained 模型。

In [4]:
#Get the path for pre-trained models and example images
if is_test():
    models_dir = os.path.join(test_data_dir, "PretrainedModels")
    image_dir = os.path.join(test_data_dir, "Images")
else:
    models_dir = os.path.join(data_dir, "PretrainedModels")
    if not os.path.exists(models_dir):
        os.makedirs(models_dir)
    
    image_dir = os.path.join(data_dir, "Images")
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
        
print("Model directory", models_dir)
print("Image directory", image_dir)

Model directory C:\Data\CNTKTestData\Tutorials\data\BerkeleySegmentationDataset\PretrainedModels
Image directory C:\Data\CNTKTestData\Tutorials\data\BerkeleySegmentationDataset\Images


In [5]:
if not os.path.isfile(os.path.join(models_dir, "VDSR.model")):
    print("Downloading VDSR model...")
    urlretrieve("https://www.cntk.ai/Models/SuperResolution/VDSR.model", os.path.join(models_dir, "VDSR.model"))
else:
    print("Using cached VDSR model")

if not os.path.isfile(os.path.join(models_dir, "DRNN.model")):
    print("Downloading DRNN model...")
    urlretrieve("https://www.cntk.ai/Models/SuperResolution/DRNN.model", os.path.join(models_dir, "DRNN.model"))
else:
    print("Using cached DRNN.model")
    
if not os.path.isfile(os.path.join(models_dir, "SRResNet.model")):
    print("Downloading SRResNet model...")
    urlretrieve("https://www.cntk.ai/Models/SuperResolution/SRResNet.model", os.path.join(models_dir, "SRResNet.model"))
else:
    print("Using cached SRResNet.model")

if not os.path.isfile(os.path.join(models_dir, "SRGAN.model")):
    print("Downloading SRGAN model...")
    urlretrieve("https://www.cntk.ai/Models/SuperResolution/SRGAN.model", os.path.join(models_dir, "SRGAN.model"))
else:
    print("Using cached SRGAN model")

print("Loading pretrained models...")
VDSR_model = C.load_model(os.path.join(models_dir, "VDSR.model"))
DRNN_model = C.load_model(os.path.join(models_dir, "DRNN.model"))
SRResNet_model = C.load_model(os.path.join(models_dir, "SRResNet.model"))
SRGAN_model = C.load_model(os.path.join(models_dir, "SRGAN.model"))

models = [VDSR_model, DRNN_model, SRResNet_model, SRGAN_model]

print("Loaded pretrained models.")

Using cached VDSR model
Using cached DRNN.model
Using cached SRResNet.model
Using cached SRGAN model
Loading pretrained models...
Loaded pretrained models.


_unchecked_

我们将采取一个例子的形象, 我们想尝试我们的模型。

In [6]:
from shutil import copyfile

if not os.path.isfile(os.path.join(image_dir, "253027.jpg")):
    print("Downloading example image ...")
    link = "https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/segbench/BSDS300/html/images/plain/normal/color/253027.jpg"
    urlretrieve(link, os.path.join(example_folder, "253027.jpg"))
else:
    print("Using cached image file")
    copyfile(os.path.join(image_dir, "253027.jpg"), os.path.join(example_folder, "253027.jpg"))

Using cached image file


_unchecked_

我们也将保存一份次的插值效果的每一个图像, 我们正在测试, 只是为了参考。

In [7]:
save_folder = os.path.join(results_folder, "bicubic")

#upscale by bicubic and save for reference
for entry in os.listdir(example_folder):
    filename = os.path.join(example_folder, entry)
        
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    
    img = Image.open(filename)
    out = img.resize((2 * img.width, 2 * img.height), Image.BICUBIC)
    out.save(os.path.join(save_folder, entry))

_unchecked_

<p>现在我们终于准备好评估我们的模型了。下面的代码将所有图像的所有可能的过滤器组合在一起。<code>evaluate</code>的调用中的参数是根据所讨论的模型设置的。例如, 索引0和1上的模型只学习残差图像, 因此我们设置了 <code>residual_model = True</code> 。</p>

<p>此外, 所有推广都是在筛选器的第一个元素中完成的。因此, 只有<b>VDSR</b>和<b>DRNN</b>可以是筛选器的第二个元素, 并且在这种情况下必须跳过它们的预处理部分。我们处理这些图像并将它们保存在适当的文件夹中。文件夹名称反映了使用的筛选器。有关其他注释, 请参见下面的代码。</p>

In [8]:
#loop thorugh every model
for i in range(4):
    save_folder = os.path.join(results_folder, model_names[i] + "_results")
        
    #loop through every image in example_folder
    for entry in os.listdir(example_folder):
        filename = os.path.join(example_folder, entry)
            
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
                
        outfile = os.path.join(save_folder, entry)
            
        print("Now creating: " + outfile)
            
        #function calls for different models
        if(i < 2):
            #residual learning, image is pre-upscaled and then cleared up
            evaluate(filename, models[i], outfile, output_dims[i], pre_upscale = True, clear_up = True, residual_model = True)
        else:
            #all upscaling is within the model
            evaluate(filename, models[i], outfile, output_dims[i], pre_upscale = False, clear_up = False, residual_model = False)
      
    #loop through models which can additionally clear up image after we increased it (DRNN and VDSR)
    for j in range(2):
        #loop through results of previously applied model
        for entry in os.listdir(save_folder):
            filename = os.path.join(save_folder, entry)
            filter_folder = os.path.join(results_folder, model_names[j] + "_" + model_names[i] + "_results")
                
            if not os.path.exists(filter_folder):
                os.makedirs(filter_folder)
                
            outfile = os.path.join(filter_folder, entry)
                
            print("Now creating: " + outfile)
                
            #additionally clear up image without pre-magnifying
            evaluate(filename, models[j], outfile, output_dims[j], pre_upscale = False, clear_up = True, residual_model = True)

Now creating: data\BerkeleySegmentationDataset\example_results\VDSR_results\253027.jpg
Now creating: data\BerkeleySegmentationDataset\example_results\VDSR_VDSR_results\253027.jpg
Now creating: data\BerkeleySegmentationDataset\example_results\DRNN_VDSR_results\253027.jpg
Now creating: data\BerkeleySegmentationDataset\example_results\DRNN_results\253027.jpg
Now creating: data\BerkeleySegmentationDataset\example_results\VDSR_DRNN_results\253027.jpg
Now creating: data\BerkeleySegmentationDataset\example_results\DRNN_DRNN_results\253027.jpg
Now creating: data\BerkeleySegmentationDataset\example_results\SRResNet_results\253027.jpg


c:\repos\cntk\bindings\python\cntk\core.py:82: RuntimeWarning: data is not C contiguous; rearrange your data/computation to avoid costly data conversions
  RuntimeWarning)


Now creating: data\BerkeleySegmentationDataset\example_results\VDSR_SRResNet_results\253027.jpg
Now creating: data\BerkeleySegmentationDataset\example_results\DRNN_SRResNet_results\253027.jpg
Now creating: data\BerkeleySegmentationDataset\example_results\SRGAN_results\253027.jpg
Now creating: data\BerkeleySegmentationDataset\example_results\VDSR_SRGAN_results\253027.jpg
Now creating: data\BerkeleySegmentationDataset\example_results\DRNN_SRGAN_results\253027.jpg


_unchecked_

## 结果分析及未来工作

下面是我们能够得到的例子。没有数学公式可以评估我们的模型表现如何好。最好和最常用的方法是通过意见评分, 即向人们展示生成的图像, 这样他们就可以从1到5的评分, 这取决于他们的实际外观。

In [9]:
from IPython.display import Image
Image(url = "https://superresolution.blob.core.windows.net/superresolutionresources/example.PNG")

_unchecked_

对于更详细的分析, 我们将展示一些原始图像 (低分辨率), 突出其中的一小部分, 看看不同的过滤器是如何执行的, 以及我们能得到什么结果。我们将看到, 在一些例子中, 我们完成了我们的目标相当不错, 但并不总是。

In [10]:
Image(url = "https://superresolution.blob.core.windows.net/superresolutionresources/analysis_zebra.PNG")

In [11]:
Image(url = "https://superresolution.blob.core.windows.net/superresolutionresources/analysis_village.PNG")

In [12]:
Image(url = "https://superresolution.blob.core.windows.net/superresolutionresources/analysis_pot.PNG")

_unchecked_

<p>对于<b>将来的工作</b>, 最好尝试从<b>SRGAN</b>模型获得更好的结果。关键问题是为不同的损失函数寻找合适的系数。在我们的大部分尝试中, 我们最终会得到完全无法使用和无法辨认的图像。经过一番努力, 我们得到了体面的结果, 有更多的改进余地。</p>

<p>创建模型组合的想法<b>(筛选器)</b>为我们提供了相当好的结果。可见, 两个模型的组合结果通常比仅一个模型更能得到更好的结果。</p>